In [1]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath("../../ml/"))

from dev_wtp_io_utils import *

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
import pandas as pd

In [3]:
nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"

In [17]:
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

In [4]:
test_train_df = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")

In [5]:
train_subjs = test_train_df.loc[test_train_df.SplitGroup=='Train','sub_label'].tolist()

In [6]:
def get_Brain_Data_for_sub(sub_label,behavdesign,betaseries_path='/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/'):
    return(dev_wtp_io_utils.get_Brain_Data_betas_for_sub(sub_label,behavdesign,betaseries_path))

In [7]:
def import_sst_cond_w1_subjs_to_pkl(subjs,first_level_fileid, behavioral_design,out_folder = '../data/'):
    sst_wt_repo = '/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/WTP/wave1/'
    first_level_path = sst_wt_repo + first_level_fileid + "/"
    print(first_level_path)
    subj_count = len(subjs)
    
    subjs.sort()
    bd_dict={}
    #get the brain data from the beta files
    for sl in subjs:
        print(sl)
        try: 
            bd = get_Brain_Data_betas_for_sub(
                sl, behavioral_design,
                betaseries_path = first_level_path
            )
            bd_dict[sl]=bd
        except BehavioralDataNotFoundForBrainDataException:
            print("couldn't get data for subject " + sl + " because there was no matching behavioral data")


    #now append into a concatenated brain data file
    Brain_Data_allsubs = list(bd_dict.values())[0]
    for i in range(1,len(bd_dict.values())):
        print(i)
        val_to_append = list(bd_dict.values())[i]
        Brain_Data_allsubs= Brain_Data_allsubs.append(val_to_append)

    #dump
    out_filepath = (
        '../data/Brain_Data_' +
        #'betaseries_' +
        first_level_fileid + '_' + str(subj_count) + 'subs.pkl'
    )
    print(out_filepath)

    with open(out_filepath, 'wb') as pkl_file:
        pickle.dump(Brain_Data_allsubs,pkl_file)
        

In [34]:
wtp_data = pd.read_csv("/gpfs/projects/sanlab/bsmith16/data/wtpw1_behavdesign_clean.csv")

In [35]:
wtp_data

,Unnamed: 0,event_id,isi_pre,onset,duration,food_pic,food_num,cond,health_cond,liking_cond,liking_rating,response,isi_post,end,run,wave,subject
0,0,1,3.000000,6.114502,6.558085,MacNCheese.bmp,1,unhealthy_liked,unhealthy,liked,3,5.0,3.000000,12.672587,run1,2,DEV040
1,1,2,3.000000,18.713320,6.532640,Strawberries.bmp,2,healthy_liked,healthy,liked,4,7.0,1.780283,25.245960,run1,2,DEV040
2,2,3,1.780283,30.069421,6.519313,RedApple.bmp,3,healthy_liked,healthy,liked,4,6.0,0.095117,36.588734,run1,2,DEV040
3,3,4,0.095117,39.713532,6.532611,Fritos.bmp,4,unhealthy_disliked,unhealthy,disliked,1,5.0,2.680966,46.246143,run1,2,DEV040
4,4,5,2.680966,51.967374,6.536582,ruffles_cheddar.bmp,5,unhealthy_liked,unhealthy,liked,4,NaN,0.319097,58.503956,run1,2,DEV040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45771,27,28,1.000000,206.375708,6.532626,Dots_one.bmp,28,unhealthy_disliked,unhealthy,disliked,1,2.0,1.000000,212.908334,run2,3,DEV138
45772,28,29,1.000000,213.991678,6.533641,VeggieStraws.bmp,29,healthy_disliked,healthy,disliked,1,2.0,1.000000,220.525319,run2,3,DEV138
45773,29,30,1.000000,221.592464,6.532627,cranberries.bmp,30,healthy_disliked,healthy,disliked,1,1.0,1.000000,228.125091,run2,3,DEV138
45774,30,31,1.000000,229.192518,6.532579,Peach.bmp,31,healthy_liked,healthy,liked,3,3.0,1.000000,235.725096,run2,3,DEV138


In [27]:
sst_behavdesign_clean= pd.read_csv(ml_data_folderpath +"/" + "all_sst_events.csv")
sst_1_behavdesign_clean = wtp_behavdesign_clean[sst_behavdesign_clean.wave=='wave1']

In [28]:
sst_1_behavdesign_clean

,onset,duration,trial_type,subject,wave
0,0.00000,2.25834,failed-stop,DEV033,wave1
1,2.25834,0.50000,NaN,DEV033,wave1
2,2.75834,0.50392,correct-go,DEV033,wave1
3,4.76390,0.75000,NaN,DEV033,wave1
4,5.51390,0.47787,correct-go,DEV033,wave1
...,...,...,...,...,...
95739,414.45072,1.75000,NaN,DEV116,wave1
95740,416.20072,2.51110,correct-stop,DEV116,wave1
95741,418.71182,2.50000,NaN,DEV116,wave1
95742,421.21182,0.70028,correct-go,DEV116,wave1


In [29]:
behavdesign = sst_1_behavdesign_clean
sst_1_behavdesign_clean_dev005 = behavdesign[[s in 'DEV005' for s in behavdesign.subject]]

In [30]:
sst_1_behavdesign_clean_dev005

,onset,duration,trial_type,subject,wave
12288,0.00000,2.25834,correct-stop,DEV005,wave1
12289,2.25834,0.50000,NaN,DEV005,wave1
12290,2.75834,0.40082,correct-go,DEV005,wave1
12291,4.76390,0.75000,NaN,DEV005,wave1
12292,5.51390,0.66191,correct-go,DEV005,wave1
...,...,...,...,...,...
12539,412.82572,1.75000,NaN,DEV005,wave1
12540,414.57572,0.49767,correct-go,DEV005,wave1
12541,417.08682,0.75000,NaN,DEV005,wave1
12542,417.83682,1.50000,failed-stop,DEV005,wave1


In [31]:
sst_1_behavdesign_clean_dev005.groupby('trial_type').count()

,onset,duration,subject,wave
trial_type,,,,
correct-go,80,80,80,80
correct-stop,5,5,5,5
failed-go,16,16,16,16
failed-stop,27,27,27,27


UP TO HERE: WE NEED TO SOMEHOW MATCH THE EXPECTED BETAS.
    
THE WTP CODE LOADED DATA FROM https://docs.google.com/spreadsheets/d/1NywNwiRxwlAOjb2eZ7q58MklRWUGvgE5hlaILurwFIs/edit#gid=314762377 
    
TAHT WAS USED TO GENERATE A LIST OF BETAS AND THEN WE MATCHED TO THAT.

I THINK THIS TIME I BETTER UNDERSTAND THE LINKING BETWEEN THE BEHAVIORAL BETA DATA AND THE DATA THAT GENERATED TEH NII BETAS IN THE FIRST PLACE.

SO WE CAN JUST LINK THEM BY THE AWARENESS OF HOW THEY MATCH UP WITH EACH OTHER.

NEED TO GO BACK AND CONFIRM HOW THE GENERATED BETAS IN CONDITIONS MATCH THE TRIALS, THEN WORK FROM THERE.

In [22]:
import_sst_cond_w1_subjs_to_pkl(train_subjs[0:5],'conditions',behavioral_design=sst_1_behavdesign_clean)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/WTP/wave1/conditions/
DEV001
couldn't get data for subject DEV001 because there was no matching behavioral data
DEV005


Exception: For DEV005, expected 64 beta events but found 256

In [13]:
wtp_1_behavdesign_clean

,Unnamed: 0,event_id,isi_pre,onset,duration,food_pic,food_num,cond,health_cond,liking_cond,liking_rating,response,isi_post,end,run,wave,subject
